# ANALYSING BOND FUTURES CONTRACTS

In this notebook I analyse a CME bond futures contract

In [1]:
import datetime as dt

In [2]:
import pandas as pd

In [3]:
from financepy.products.bonds import *
from financepy.utils import *

## Example from CME

https://www.cmegroup.com/education/files/understanding-treasury-futures.pdf

I am looking at Table 3 on Page 20

In [4]:
first_delivery_dt = Date(1, 12, 2017)
last_delivery_dt = Date(28, 12, 2017)

contract_size = 100000
contract_cpn = 0.06

# December 2017 contract
bond_fut_contract = BondFuture("TYZ7", first_delivery_dt, last_delivery_dt,
                                   contract_size, contract_cpn)

In [5]:
print(bond_fut_contract)

OBJECT TYPE: BondFuture
TICKER NAME: TYZ7
FIRST DELIVERY DATE: 01-DEC-2017
LAST DELIVERY DATE: 28-DEC-2017
CONTRACT SIZE: 100000
COUPON: 0.06



In [6]:
freq = FrequencyTypes.SEMI_ANNUAL
basis = DayCountTypes.ACT_ACT_ICMA
settle_dt = Date(10, 10, 2017)
issue_dt = Date(1, 1, 2000) # Dummy

In [7]:
bonds = []
prices = []
bond = Bond(issue_dt, Date(15, 8, 2027), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 1 / 32)
bond = Bond(issue_dt, Date(15, 5, 2027), 0.02375, freq, basis); bonds.append(bond); prices.append(100 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2027), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 11, 2026), 0.02, freq, basis); bonds.append(bond); prices.append(97 + 7 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(5, 8, 2026), 0.015, freq, basis); bonds.append(bond); prices.append(93 + 14 / 32)
bond = Bond(issue_dt, Date(15, 5, 2026), 0.01625, freq, basis); bonds.append(bond); prices.append(94 + 21 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2026), 0.01625, freq, basis); bonds.append(bond); prices.append(94 + 29 / 32)
bond = Bond(issue_dt, Date(15, 11, 2025), 0.0225, freq, basis); bonds.append(bond); prices.append(99 + 25 / 32)
bond = Bond(issue_dt, Date(15, 8, 2025), 0.02, freq, basis); bonds.append(bond); prices.append(98 + 3 / 32)
bond = Bond(issue_dt, Date(15, 5, 2025), 0.02125, freq, basis); bonds.append(bond); prices.append(99 + 5 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 2, 2025), 0.02, freq, basis); bonds.append(bond); prices.append(98 + 14 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 11, 2024), 0.0225, freq, basis); bonds.append(bond); prices.append(100 + 9 / 32 + 1 / 64)
bond = Bond(issue_dt, Date(15, 8, 2024), 0.02375, freq, basis); bonds.append(bond); prices.append(101 + 7 / 32 + 1 / 64)
# bond = Bond(issue_dt, Date(15, 8, 2024), 0.01875, freq, basis); bonds.append(bond); prices.append(98 + 1 / 32 + 0/64) #TYPO IN REPT

Calculate the bond yield

In [8]:
# Get the Invoice Prices
futures_price = 125 + 8/32 + 1/64

Get the conversion factors

In [9]:
data = []

for bond, clean_price in zip(bonds, prices):
    yld = bond.yield_to_maturity(settle_dt, clean_price)
    dt = bond.maturity_dt
    cf = bond_fut_contract.conversion_factor(bond)
    pip = bond_fut_contract.principal_invoice(bond, futures_price)

    data.append({
        "Coupon": bond.cpn * 100,
        "Maturity": dt,
        "Yield": yld * 100,
        "Clean Price": clean_price,
        "TCF": cf,
        "PIP": pip,
    })

# Create the DataFrame
df = pd.DataFrame(data)

# Optional: Format the DataFrame for display
df = df.round({
    "Coupon (%)": 5,
    "Yield to Maturity (%)": 8,
    "Clean Price": 5,
    "Conversion Factor": 5,
    "PIP": 2
})

# Display the DataFrame
print(df)

    Coupon     Maturity     Yield  Clean Price     TCF        PIP
0    2.250  15-AUG-2027  2.360500     99.03125  0.7314   91619.28
1    2.375  15-MAY-2027  2.354632    100.17188  0.7455   93385.52
2    2.250  15-FEB-2027  2.348850     99.17188  0.7421   92959.62
3    2.000  15-NOV-2026  2.339005     97.23438  0.7308   91544.12
4    1.500  05-AUG-2026  2.326953     93.43750  0.7038   88161.95
5    1.625  15-MAY-2026  2.311669     94.67188  0.7186   90015.88
6    1.625  15-FEB-2026  2.298793     94.90625  0.7252   90842.63
7    2.250  15-NOV-2025  2.279461     99.78125  0.7703   96492.11
8    2.000  15-AUG-2025  2.266160     98.09375  0.7612   95352.19
9    2.125  15-MAY-2025  2.243883     99.17188  0.7749   97068.33
10   2.000  15-FEB-2025  2.229036     98.45312  0.7741   96968.12
11   2.250  15-NOV-2024  2.204292    100.29688  0.7943   99498.49
12   2.375  15-AUG-2024  2.179562    101.23438  0.8072  101114.41


In [10]:
ctd = bond_fut_contract.ctd(bonds, prices, futures_price)
print("CTD MATURITY", "CTD COUPON")
print(str(ctd.maturity_dt), ctd.cpn*100)

CTD MATURITY CTD COUPON
15-AUG-2027 2.25


Copyright (c) 2019, Dominic O'Kane 